In [28]:
import sys
sys.path.append('../')

import os
from glob import glob
from tqdm import tqdm

from utils.elastix import excute_cmd, register_elastix, control_points_transformix
from utils.filemanager import create_directory_if_not_exists, replace_text_in_file

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
print(excute_cmd('elastix --help'))

elastix version: 4.200

elastix registers a moving image to a fixed image.
The registration-process is specified in the parameter file.

Call elastix from the command line with mandatory arguments:
-f        fixed image
-m        moving image
-out      output directory
-p        parameter file, elastix handles 1 or more "-p"

Optional extra commands:
-fMask    mask for fixed image
-mMask    mask for moving image
-t0       parameter file for initial transform
-priority set the process priority to high or belownormal (Windows only)
-threads  set the maximum number of threads of elastix

The parameter-file must contain all the information necessary for elastix to run properly. That includes which metric to use, which optimizer, which transform, etc.
It must also contain information specific for the metric, optimizer, transform, ...
For a usable parameter-file, see the website.

Need further help? Check the website http://elastix.isi.uu.nl, or mail elastix.support@gmail.com.



In [2]:
os.listdir('../niftiData/')

['copd1', 'copd2', 'copd3', 'copd4']

In [5]:
train_path = '../niftiData'

# prepare the paths
exhale_volumes = [path.replace('\\', '/') for path in sorted(glob(os.path.join(train_path, "***" , "*eBHCT.nii.gz"), recursive=True))]
inhale_volumes = [path.replace('\\', '/') for path in sorted(glob(os.path.join(train_path, "***" , "*iBHCT.nii.gz"), recursive=True))]

In [8]:
exhale_volumes

['../niftiData/copd1/copd1_eBHCT.nii.gz',
 '../niftiData/copd2/copd2_eBHCT.nii.gz',
 '../niftiData/copd3/copd3_eBHCT.nii.gz',
 '../niftiData/copd4/copd4_eBHCT.nii.gz']

Register and transform all control points

In [15]:
os.listdir('../elastix-parameters')

['Par0003', 'Par0004', 'Par0007', 'Par0011']

As we have alot of parameters that are suitable for CT registration. We can experiment them and see which performs the best.

Experimenting Par0003 models with elastix and transformix.

In [18]:
Par0003_base = '../elastix-parameters/Par0003'
Par0003_base

'../elastix-parameters/Par0003'

In [19]:
os.listdir(Par0003_base)

['404px-Par0003screenshot1.png',
 'Par0003.affine.txt',
 'Par0003.bs-R1-fg.txt',
 'Par0003.bs-R1-ug.txt',
 'Par0003.bs-R2-fg.txt',
 'Par0003.bs-R2-ug.txt',
 'Par0003.bs-R3-fg.txt',
 'Par0003.bs-R3-ug.txt',
 'Par0003.bs-R4-fg.txt',
 'Par0003.bs-R4-ug.txt',
 'Par0003.bs-R5-fg.txt',
 'Par0003.bs-R5-ug.txt',
 'Par0003.bs-R6-fg.txt',
 'Par0003.bs-R6-ug.txt',
 'Par0003.bs-R7-fg.txt',
 'Par0003.bs-R7-ug.txt',
 'Par0003.bs-R8-fg.txt',
 'Par0003.bs-R8-ug.txt',
 'README.md']

In [29]:
reg_params = f'-p "{os.path.join(Par0003_base, "Par0003.affine.txt")}"'.replace('\\', '/')
reg_params

'-p "../elastix-parameters/Par0003/Par0003.affine.txt"'

In [31]:
for e_path, i_path in tqdm(zip(exhale_volumes, inhale_volumes)):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # get control points path from rawData dir
    e_cntl_pt = f'../rawData/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'

    '''
    elastix is started at Mon Dec 04 20:30:00 2023.
    
    -------------------------------------------------------------------------
    
    Running elastix with parameter file 0: "../elastix-parameters/Par0003/Par0003.affine.txt".
    
    Current time: Mon Dec 04 20:30:00 2023.
    Reading the elastix parameters from file ...
    
    Installing all components.
    InstallingComponents was successful.
    
    ELASTIX version: 4.200
    Command line options from ElastixBase:
    -f        ../niftiData/copd1/copd1_iBHCT.nii.gz
    -m        ../niftiData/copd1/copd1_eBHCT.nii.gz
    -fMask    unspecified, so no fixed mask used
    -mMask    unspecified, so no moving mask used
    -out      output/images/output_copd1_iBHCT/copd1_eBHCT/
    -p        ../elastix-parameters/Par0003/Par0003.affine.txt
    -priority unspecified, so NORMAL process priority
    -threads  unspecified, so all available threads are used
    Command line options from TransformBase:
    -t0       unspecified, so no initial transform used
    
    Reading images...
    Reading images took 882 ms.
    
    WARNING: the fixed pyramid schedule is not fully specified!
      A default pyramid schedule is used.
    WARNING: the moving pyramid schedule is not fully specified!
      A default pyramid schedule is used.
    WARNING: The parameter "AutomaticTransformInitializationMethod", requested at entry number 0, does not exist at all.
      The default value "GeometricalCenter" is used instead.
    Scales are estimated automatically.
    Scales for transform parameters are: [8964.941406, 8964.941406, 9375.000000, 8964.941406, 8964.941406, 9375.000000, 8964.941406, 8964.941406, 9375.000000, 1.000000, 1.000000, 1.000000]
    Initialization of all components (before registration) took: 32 ms.
    
    itk::ExceptionObject (017EF938)
    Location: "ElastixTemplate - Run()"
    File: d:\tk\itk\3.14\src\code\common\itkImportImageContainer.txx
    Line: 188
    Description: Failed to allocate memory for image.
    
    Error occurred during actual registration.
    
    
    Errors occurred!
    '''

    # elastix registration
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        # fMask = test_mask,
        reg_params = reg_params,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # # transformix control point transformation
    # control_points_transformix(
    #     fixed_path = i_path, 
    #     moving_path = e_path, 
    #     input_points = e_cntl_pt,
    #     transform_path = f'output/images/output_{sample_name}/{e_filename_full}/TransformParameters.1.txt',
    #     replace_text_in_file_callback = replace_text_in_file,
    #     create_dir_callback = create_directory_if_not_exists, 
    #     excute_cmd_callback = excute_cmd)

    break
    

0it [00:03, ?it/s]

Command failed with an error: elastix -f "../niftiData/copd1/copd1_iBHCT.nii.gz" -m "../niftiData/copd1/copd1_eBHCT.nii.gz" -p "../elastix-parameters/Par0003/Par0003.affine.txt" -out "output/images/output_copd1_iBHCT/copd1_eBHCT"

